# DP and DPLL with various heuristics comparision

author: Witalis Domitrz <witekdomitrz@gmail.com>

## Types

In [0]:
from typing import MutableSet, FrozenSet, Iterable, List
# I represent the CNF formula as a frozenset of frozen sets
Literal = int
Clause = FrozenSet[Literal]
CNF = FrozenSet[Clause]

# Helper Functions
def to_list(cnf: CNF) -> List[List[Literal]]:
    return list(map(list, cnf))

def from_lists(cnf: Iterable[Iterable[Literal]]) -> CNF:
    return frozenset(map(frozenset, cnf))

## Base algorithm

In [0]:
from typing import Tuple, Callable
class BaseAlgorithm:
    @staticmethod
    def is_tautology(c: Clause) -> bool:
        return not c.isdisjoint(frozenset(map(lambda l: -l, c)))

    @staticmethod
    def get_singleton_literals(cnf: CNF) -> FrozenSet[Literal]:
        literals = set()
        for c in cnf:
            if len(c) == 1:
                literals |= c
        return frozenset(literals)
    
    @classmethod
    def tatuology_rule(cls, cnf: CNF) -> Tuple[CNF, bool]:
        # We return False, because the tatutology rule is the first one, so
        # there is no point in trying it again if only it succeded
        return frozenset(filter(lambda c: not cls.is_tautology(c), cnf)), False

    @classmethod
    def unit_clause_rule(cls, cnf: CNF) -> Tuple[CNF, bool]:
        singleton_literals = cls.get_singleton_literals(cnf)
        # I process all the singleton literals at once. I am awere that is is
        # different than the classical iteration and it can be easilly changed
        # if needed.
        # To process all the singleton literals at once we cannot process l 
        # and -l at the same time.
        singleton_literals = frozenset(filter(
            lambda l: -l not in singleton_literals or l > 0, singleton_literals))
        neg_singleton_literals = frozenset(map(lambda l: -l, singleton_literals))
        if singleton_literals:
            return frozenset(c - neg_singleton_literals for c in cnf if 
                             c.isdisjoint(singleton_literals)), True
        else:
            return cnf, False

    @classmethod
    def pure_literal_rule(cls, cnf: CNF) -> Tuple[CNF, bool]:
        # union needs nonzero number of parameters
        if not cnf:
            return cnf, True
        # I process all one_sign_literals at once. I am awere that it is
        # different than the classical iteration and it can be easilly changed
        # if needed.
        all_literals = frozenset.union(*cnf)
        one_sign_literals = frozenset(
            filter(lambda l: -l not in all_literals, all_literals))
        if one_sign_literals:
            return frozenset(c for c in cnf if c.isdisjoint(one_sign_literals)), True
        else:
            return cnf, False

    @classmethod
    def simplify(cls, cnf: CNF) -> Tuple[CNF, bool]:
        simplified_acc = False
        for simplifier in [cls.tatuology_rule, 
                           cls.unit_clause_rule, 
                           cls.pure_literal_rule]:
            cnf, simplified = simplifier(cnf)
            simplified_acc |= simplified
        return cnf, simplified_acc

    def run(self, cnf: CNF) -> bool:
        # return satisfiability if there are no clauses
        if not cnf:
            return True            
        # return unsatisfiability if there is an empty clause
        for c in cnf:
            if not c:
                return False
        # try to simplify formula
        cnf, simplified, = self.simplify(cnf)
        if not simplified:
            return self.core_function(cnf)
        else:
            return self.run(cnf)

    @staticmethod
    def parse_preambule(preambule: str) -> Tuple[int, int]:
        preambule =  preambule.strip().split()
        line_start, format_name = preambule[0], preambule[1]
        if line_start != "p" or format_name != "cnf":
            raise NotImplementedError("File format not supported: " + format_name)
        no_variables, no_clauses = int(preambule[2]), int(preambule[3])
        return no_variables, no_clauses

    @classmethod
    def get_cnf_from_file(cls, fn: str) -> Tuple[int, int, CNF]:
        with open(fn) as f:
            lines = f.readlines()
        # filter comments
        lines = list(filter(lambda line: line[0] != 'c', lines))
        # Parse preambule
        no_variables, no_clauses = cls.parse_preambule(lines[0])
        # Split to clauses
        data = " ".join(lines[1:]).strip().split()
        clauses, clause, clauses_counter = set(), set(), 0
        for x in data:
            if x == "0":
                clauses.add(frozenset(clause))
                clause = set()
                clauses_counter += 1
                if clauses_counter == no_clauses:
                    break
            else:
                clause.add(int(x))
        return no_variables, no_clauses, frozenset(clauses)

    def __call__(self, fn: str) -> None:
        (_, _, cnf) = self.get_cnf_from_file(fn)
        if self.run(cnf):
            print("SAT")
        else:
            print("UNSAT")

    def core_function(self, cnf: CNF) -> bool:
        raise NotImplemented("This is an abstract class")

class AlgorithmWithLiteralChoosingHeuristic(BaseAlgorithm):
    def __init__(self, literal_heuristic: Callable[[CNF], Literal]) -> None:
        self.literal_heuristic = literal_heuristic

    def core_function(self, cnf: CNF) -> bool:
        l = self.literal_heuristic(cnf)
        return self.core_function_with_literal_heurstic(l, cnf)
    
    def core_function_with_literal_heurstic(self, l: Literal, cnf: CNF) -> bool:
        raise NotImplemented("This is an abstract class")

## The algoritms

### DP algorithm

In [0]:
class DP(AlgorithmWithLiteralChoosingHeuristic):
    def __init__(self, literal_heuristic: Callable[[CNF], Literal]) -> None:
        super().__init__(literal_heuristic)

    def core_function_with_literal_heurstic(self, l: Literal, cnf: CNF) -> bool:
        l_and_neg_l = frozenset({l, -l})
        c_with_l = frozenset(c - l_and_neg_l for c in cnf if l in c)
        c_with_neg_l = frozenset(c - l_and_neg_l for c in cnf if -l in c)
        resolvents = frozenset(c1.union(c2) for c1 in c_with_l for c2 in c_with_neg_l)
        cnf = resolvents.union(filter(l_and_neg_l.isdisjoint, cnf))
        return self.run(cnf)

### DPLL algorithm

In [0]:
class DPLL(AlgorithmWithLiteralChoosingHeuristic):
    def __init__(self, literal_heuristic: Callable[[CNF], Literal]) -> None:
        super().__init__(literal_heuristic)

    def core_function_with_literal_heurstic(self, l: Literal, cnf: CNF) -> bool:
        if self.run(cnf | frozenset({frozenset({l})})):
            return True
        return self.run(cnf | frozenset({frozenset({-l})}))

### Selected algorithms

In [0]:
algorithms = {"DPLL": DPLL, "DP": DP}

## Heuristics for choosing a literal

### A helper function

In [0]:
from typing import Dict
# Helper functions
def count_literals(cnf: CNF) -> Dict[Literal, int]:
    counts = {}
    for c in cnf:
        for l in c:
            counts[l] = counts.get(l, 0) + 1
    return counts

### The heuristics implementation

In [0]:
def first_of_first(cnf: CNF) -> Literal:
    for c in cnf:
        for l in c:
            return l

def most_common(cnf: CNF) -> Literal:
    counts = count_literals(cnf)
    return max(counts.items(), key=lambda x: x[1])[0]

def least_common(cnf: CNF) -> Literal:
    counts = count_literals(cnf)
    return min(counts.items(), key=lambda x: x[1])[0]

def middle_common(cnf: CNF) -> Literal:
    counts = count_literals(cnf)
    # It is possible to do is in O(n) time, but here is O(n * log(n)) solution
    return sorted(counts.items(), key=lambda x: x[1])[len(counts) // 2][0]

### Selected heuristics

In [0]:
heuristics = {"first_of_first": first_of_first, 
              "most_common": most_common, 
              "least_common": least_common, 
              "middle_common": middle_common}

## Comparision

### Helper functions

In [0]:
import time
import signal

def measure_time_on_file_with_limit(fn: str, 
                                    algorithm: BaseAlgorithm, 
                                    time_limit_in_seconds: int) -> Tuple[bool, float]:
    def limit_handler(x, y):
        raise Exception("eot")
    (_, _, cnf) = algorithm.get_cnf_from_file(fn)
    result = None
    signal.signal(signal.SIGALRM, limit_handler)
    start = time.time()
    signal.alarm(time_limit_in_seconds)
    try:
        result = algorithm.run(cnf)
    except Exception:
        # There was a timeout iff result is None
        pass
    end = time.time()
    signal.alarm(0)
    return (result, end - start)

### Combinations to compare

In [0]:
algorithms_with_heuristics =  {
    algorithm_name + "_" + heuristic_name : algorithm(heuristic)
    for algorithm_name, algorithm in algorithms.items()
    for heuristic_name, heuristic in heuristics.items()}

In [0]:
files_to_test = ["aim-50-1_6-no-1.cnf", "aim-50-3_4-yes1-1.cnf", "aim-50-2_0-no-4.cnf", "aim-50-2_0-yes1-1.cnf", "uf20-0101.cnf", "uf50-0707.cnf", "uuf50-0707.cnf", "aim-100-2_0-no-1.cnf", "aim-100-2_0-no-2.cnf", "uf125-012.cnf", "uuf125-012.cnf"]

In [0]:
!wget https://www.cs.ubc.ca/~hoos/SATLIB/Benchmarks/SAT/RND3SAT/uf20-91.tar.gz
!tar xf uf20-91.tar.gz
!rm uf20-91.tar.gz
!wget https://www.cs.ubc.ca/~hoos/SATLIB/Benchmarks/SAT/RND3SAT/uf50-218.tar.gz
!tar xf uf50-218.tar.gz
!rm uf50-218.tar.gz
!wget https://www.cs.ubc.ca/~hoos/SATLIB/Benchmarks/SAT/RND3SAT/uuf50-218.tar.gz
!tar xf uuf50-218.tar.gz
!rm uuf50-218.tar.gz
!wget https://www.cs.ubc.ca/~hoos/SATLIB/Benchmarks/SAT/RND3SAT/uf125-538.tar.gz
!tar xf uf125-538.tar.gz
!rm uf125-538.tar.gz
!wget https://www.cs.ubc.ca/~hoos/SATLIB/Benchmarks/SAT/RND3SAT/uuf125-538.tar.gz
!tar xf uuf125-538.tar.gz
!rm uuf125-538.tar.gz
!wget https://www.cs.ubc.ca/~hoos/SATLIB/Benchmarks/SAT/DIMACS/AIM/aim.tar.gz
!tar xf aim.tar.gz
!rm aim.tar.gz
!find -name "*.cnf" -exec mv \{\} ./ \; 2> /dev/null
!mkdir -p tests
!mv "aim-50-1_6-no-1.cnf" "aim-50-3_4-yes1-1.cnf" "aim-50-2_0-no-4.cnf" "aim-50-2_0-yes1-1.cnf" "uf20-0101.cnf" "uf50-0707.cnf" "uuf50-0707.cnf" "aim-100-2_0-no-1.cnf" "aim-100-2_0-no-2.cnf" "uf125-012.cnf" "uuf125-012.cnf" ./tests/
!rm *.cnf
!cp tests/* ./

In [0]:
results = {}

In [0]:
for f in files_to_test:
    for n, a in algorithms_with_heuristics.items():
        res = measure_time_on_file_with_limit(f, a, 30)
        results[(n, f)] = res
        print(n, f, res)

DPLL_first_of_first aim-50-1_6-no-1.cnf (False, 0.32840657234191895)
DPLL_most_common aim-50-1_6-no-1.cnf (False, 0.10799908638000488)
DPLL_least_common aim-50-1_6-no-1.cnf (None, 30.000643968582153)
DPLL_middle_common aim-50-1_6-no-1.cnf (False, 10.321396589279175)
DP_first_of_first aim-50-1_6-no-1.cnf (None, 30.399317502975464)
DP_most_common aim-50-1_6-no-1.cnf (None, 31.21528172492981)
DP_least_common aim-50-1_6-no-1.cnf (False, 0.017838716506958008)
DP_middle_common aim-50-1_6-no-1.cnf (None, 30.82250189781189)
DPLL_first_of_first aim-50-3_4-yes1-1.cnf (True, 0.03325295448303223)
DPLL_most_common aim-50-3_4-yes1-1.cnf (True, 0.5202136039733887)
DPLL_least_common aim-50-3_4-yes1-1.cnf (True, 6.6106555461883545)
DPLL_middle_common aim-50-3_4-yes1-1.cnf (True, 0.2520878314971924)
DP_first_of_first aim-50-3_4-yes1-1.cnf (None, 33.025670289993286)
DP_most_common aim-50-3_4-yes1-1.cnf (None, 31.649574995040894)
DP_least_common aim-50-3_4-yes1-1.cnf (None, 39.9102246761322)
DP_middle_com

### Results

In [0]:
print(results)

{('DPLL_first_of_first', 'aim-50-1_6-no-1.cnf'): (False, 0.32840657234191895), ('DPLL_most_common', 'aim-50-1_6-no-1.cnf'): (False, 0.10799908638000488), ('DPLL_least_common', 'aim-50-1_6-no-1.cnf'): (None, 30.000643968582153), ('DPLL_middle_common', 'aim-50-1_6-no-1.cnf'): (False, 10.321396589279175), ('DP_first_of_first', 'aim-50-1_6-no-1.cnf'): (None, 30.399317502975464), ('DP_most_common', 'aim-50-1_6-no-1.cnf'): (None, 31.21528172492981), ('DP_least_common', 'aim-50-1_6-no-1.cnf'): (False, 0.017838716506958008), ('DP_middle_common', 'aim-50-1_6-no-1.cnf'): (None, 30.82250189781189), ('DPLL_first_of_first', 'aim-50-3_4-yes1-1.cnf'): (True, 0.03325295448303223), ('DPLL_most_common', 'aim-50-3_4-yes1-1.cnf'): (True, 0.5202136039733887), ('DPLL_least_common', 'aim-50-3_4-yes1-1.cnf'): (True, 6.6106555461883545), ('DPLL_middle_common', 'aim-50-3_4-yes1-1.cnf'): (True, 0.2520878314971924), ('DP_first_of_first', 'aim-50-3_4-yes1-1.cnf'): (None, 33.025670289993286), ('DP_most_common', 'ai

In [0]:
results_backup = {('DPLL_first_of_first', 'aim-50-1_6-no-1.cnf'): (False, 0.32840657234191895), ('DPLL_most_common', 'aim-50-1_6-no-1.cnf'): (False, 0.10799908638000488), ('DPLL_least_common', 'aim-50-1_6-no-1.cnf'): (None, 30.000643968582153), ('DPLL_middle_common', 'aim-50-1_6-no-1.cnf'): (False, 10.321396589279175), ('DP_first_of_first', 'aim-50-1_6-no-1.cnf'): (None, 30.399317502975464), ('DP_most_common', 'aim-50-1_6-no-1.cnf'): (None, 31.21528172492981), ('DP_least_common', 'aim-50-1_6-no-1.cnf'): (False, 0.017838716506958008), ('DP_middle_common', 'aim-50-1_6-no-1.cnf'): (None, 30.82250189781189), ('DPLL_first_of_first', 'aim-50-3_4-yes1-1.cnf'): (True, 0.03325295448303223), ('DPLL_most_common', 'aim-50-3_4-yes1-1.cnf'): (True, 0.5202136039733887), ('DPLL_least_common', 'aim-50-3_4-yes1-1.cnf'): (True, 6.6106555461883545), ('DPLL_middle_common', 'aim-50-3_4-yes1-1.cnf'): (True, 0.2520878314971924), ('DP_first_of_first', 'aim-50-3_4-yes1-1.cnf'): (None, 33.025670289993286), ('DP_most_common', 'aim-50-3_4-yes1-1.cnf'): (None, 31.649574995040894), ('DP_least_common', 'aim-50-3_4-yes1-1.cnf'): (None, 39.9102246761322), ('DP_middle_common', 'aim-50-3_4-yes1-1.cnf'): (None, 35.874491930007935), ('DPLL_first_of_first', 'aim-50-2_0-no-4.cnf'): (False, 5.642754077911377), ('DPLL_most_common', 'aim-50-2_0-no-4.cnf'): (False, 2.764810800552368), ('DPLL_least_common', 'aim-50-2_0-no-4.cnf'): (None, 30.000403881072998), ('DPLL_middle_common', 'aim-50-2_0-no-4.cnf'): (False, 5.9175732135772705), ('DP_first_of_first', 'aim-50-2_0-no-4.cnf'): (None, 31.802494764328003), ('DP_most_common', 'aim-50-2_0-no-4.cnf'): (None, 30.748937845230103), ('DP_least_common', 'aim-50-2_0-no-4.cnf'): (None, 35.09308910369873), ('DP_middle_common', 'aim-50-2_0-no-4.cnf'): (None, 33.52126431465149), ('DPLL_first_of_first', 'aim-50-2_0-yes1-1.cnf'): (True, 2.2926716804504395), ('DPLL_most_common', 'aim-50-2_0-yes1-1.cnf'): (True, 1.8174259662628174), ('DPLL_least_common', 'aim-50-2_0-yes1-1.cnf'): (None, 30.000263214111328), ('DPLL_middle_common', 'aim-50-2_0-yes1-1.cnf'): (True, 4.209372282028198), ('DP_first_of_first', 'aim-50-2_0-yes1-1.cnf'): (None, 30.84337019920349), ('DP_most_common', 'aim-50-2_0-yes1-1.cnf'): (None, 31.465791702270508), ('DP_least_common', 'aim-50-2_0-yes1-1.cnf'): (None, 35.3998601436615), ('DP_middle_common', 'aim-50-2_0-yes1-1.cnf'): (None, 32.43483066558838), ('DPLL_first_of_first', 'uf20-0101.cnf'): (True, 0.007339954376220703), ('DPLL_most_common', 'uf20-0101.cnf'): (True, 0.002201557159423828), ('DPLL_least_common', 'uf20-0101.cnf'): (True, 0.019530057907104492), ('DPLL_middle_common', 'uf20-0101.cnf'): (True, 0.0033349990844726562), ('DP_first_of_first', 'uf20-0101.cnf'): (None, 31.93820333480835), ('DP_most_common', 'uf20-0101.cnf'): (None, 32.79548263549805), ('DP_least_common', 'uf20-0101.cnf'): (True, 12.789995431900024), ('DP_middle_common', 'uf20-0101.cnf'): (None, 31.507838249206543), ('DPLL_first_of_first', 'uf50-0707.cnf'): (True, 0.2338576316833496), ('DPLL_most_common', 'uf50-0707.cnf'): (True, 0.016969919204711914), ('DPLL_least_common', 'uf50-0707.cnf'): (True, 2.466982126235962), ('DPLL_middle_common', 'uf50-0707.cnf'): (True, 0.45749759674072266), ('DP_first_of_first', 'uf50-0707.cnf'): (None, 34.73366975784302), ('DP_most_common', 'uf50-0707.cnf'): (None, 34.67880582809448), ('DP_least_common', 'uf50-0707.cnf'): (None, 34.53481483459473), ('DP_middle_common', 'uf50-0707.cnf'): (None, 33.66024613380432), ('DPLL_first_of_first', 'uuf50-0707.cnf'): (False, 0.34107398986816406), ('DPLL_most_common', 'uuf50-0707.cnf'): (False, 0.14882326126098633), ('DPLL_least_common', 'uuf50-0707.cnf'): (False, 8.376945972442627), ('DPLL_middle_common', 'uuf50-0707.cnf'): (False, 0.5267465114593506), ('DP_first_of_first', 'uuf50-0707.cnf'): (None, 36.22384071350098), ('DP_most_common', 'uuf50-0707.cnf'): (None, 36.01667642593384), ('DP_least_common', 'uuf50-0707.cnf'): (None, 35.129183292388916), ('DP_middle_common', 'uuf50-0707.cnf'): (None, 35.67857122421265), ('DPLL_first_of_first', 'aim-100-2_0-no-1.cnf'): (None, 30.000799417495728), ('DPLL_most_common', 'aim-100-2_0-no-1.cnf'): (None, 30.000829458236694), ('DPLL_least_common', 'aim-100-2_0-no-1.cnf'): (None, 30.00119972229004), ('DPLL_middle_common', 'aim-100-2_0-no-1.cnf'): (None, 30.000582218170166), ('DP_first_of_first', 'aim-100-2_0-no-1.cnf'): (None, 31.850049018859863), ('DP_most_common', 'aim-100-2_0-no-1.cnf'): (None, 32.36411428451538), ('DP_least_common', 'aim-100-2_0-no-1.cnf'): (None, 36.74852228164673), ('DP_middle_common', 'aim-100-2_0-no-1.cnf'): (None, 33.22119641304016), ('DPLL_first_of_first', 'aim-100-2_0-no-2.cnf'): (None, 30.000890493392944), ('DPLL_most_common', 'aim-100-2_0-no-2.cnf'): (None, 30.00075078010559), ('DPLL_least_common', 'aim-100-2_0-no-2.cnf'): (None, 30.00104594230652), ('DPLL_middle_common', 'aim-100-2_0-no-2.cnf'): (None, 30.00079035758972), ('DP_first_of_first', 'aim-100-2_0-no-2.cnf'): (None, 32.38468098640442), ('DP_most_common', 'aim-100-2_0-no-2.cnf'): (None, 33.61148452758789), ('DP_least_common', 'aim-100-2_0-no-2.cnf'): (None, 34.21180701255798), ('DP_middle_common', 'aim-100-2_0-no-2.cnf'): (None, 36.17365741729736), ('DPLL_first_of_first', 'uf125-012.cnf'): (None, 30.00183939933777), ('DPLL_most_common', 'uf125-012.cnf'): (True, 1.1613168716430664), ('DPLL_least_common', 'uf125-012.cnf'): (None, 30.00194025039673), ('DPLL_middle_common', 'uf125-012.cnf'): (None, 30.00209903717041), ('DP_first_of_first', 'uf125-012.cnf'): (None, 37.78161263465881), ('DP_most_common', 'uf125-012.cnf'): (None, 33.502182960510254), ('DP_least_common', 'uf125-012.cnf'): (None, 33.45947265625), ('DP_middle_common', 'uf125-012.cnf'): (None, 33.36627268791199), ('DPLL_first_of_first', 'uuf125-012.cnf'): (None, 30.0019314289093), ('DPLL_most_common', 'uuf125-012.cnf'): (None, 30.001338958740234), ('DPLL_least_common', 'uuf125-012.cnf'): (None, 30.00712299346924), ('DPLL_middle_common', 'uuf125-012.cnf'): (None, 30.001800298690796), ('DP_first_of_first', 'uuf125-012.cnf'): (None, 33.15198755264282), ('DP_most_common', 'uuf125-012.cnf'): (None, 35.49843502044678), ('DP_least_common', 'uuf125-012.cnf'): (None, 34.35980248451233), ('DP_middle_common', 'uuf125-012.cnf'): (None, 36.64634847640991)}

In [0]:
import pandas as pd
results_formated = {}
for (alg_heu, fn), (res, t) in results.items():
    [alg, heu] = alg_heu.split('_', 1)
    if (alg, heu) not in results_formated:
        results_formated[(alg, heu)] = {}
    results_formated[(alg, heu)][fn] = t if res is not None else None
res_df = pd.DataFrame(results_formated)

In [57]:
res_df

DPLL              ...           DP              
                      first_of_first most_common  ... least_common middle_common
aim-50-1_6-no-1.cnf         0.328407    0.107999  ...     0.017839          None
aim-50-3_4-yes1-1.cnf       0.033253    0.520214  ...          NaN          None
aim-50-2_0-no-4.cnf         5.642754    2.764811  ...          NaN          None
aim-50-2_0-yes1-1.cnf       2.292672    1.817426  ...          NaN          None
uf20-0101.cnf               0.007340    0.002202  ...    12.789995          None
uf50-0707.cnf               0.233858    0.016970  ...          NaN          None
uuf50-0707.cnf              0.341074    0.148823  ...          NaN          None
aim-100-2_0-no-1.cnf             NaN         NaN  ...          NaN          None
aim-100-2_0-no-2.cnf             NaN         NaN  ...          NaN          None
uf125-012.cnf                    NaN    1.161317  ...          NaN          None
uuf125-012.cnf                   NaN         NaN  ...          NaN          None

[11 rows x 8 columns]